In [ ]:
# Course link
# notebook
#  https://github.com/codebasics/llm-fine-tuning-crash-course/blob/main/2_unsloth_finetuning.ipynb

In [ ]:
# !pip install -q unsloth
# !pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 20148
dtype=None
load_in_4bit=True



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",  # "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


dtype: which specifies the model weigths and computations.

None- automatically slects the appropriate data type based on the hardware


*torch.float16-- uses 16bit floating point precision, reducing memory usage and potentially increasing speed on compatible GPUs*


**torch.bfloat16 --- similar to float16 but with a weider dynamic range, beneficial for certain hardware like NVIDIA A100 GPUs**

**load_in_4bit - determines whether to load the model using 4bit quantization . ideal for sceanrios where memory efficiency is crucial, such as deploying models on edge devices or during experimentation**


In [ ]:
# now we be using get_peft_model from unsloth FastLangugaeModel class to attach
#  adapters(peft layers) on top of the models in order to perform QLoRA


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16, # we can choose any number like 8,16,32,64,128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16, # higher alpha value assigns more weight to the LoRA activations
    lora_dropout= 0,
    bias='none',
    use_gradient_checkpointing='unsloth', # true or unsloth for very long context
    random_state = 3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.




1.   **r**: the rank of low rank metrics in LoRA; higher alues can capture more information but increase memory usage
2.   **target_modules**: list of model components where LoRA adapters are inserted for fientuning
3. **lora_alpha**; scaling factor factor for the lora updates, controls the impact of the adapters on the models outputs
4. **lora_dropout**; dropout rate applies to lora layers during training to prevent overfitting
5. **use_gradient_checkpointing**; enables gradient checkpointing to reduce the memory usage during training, unsloth use unsloths optimized version
6. **use_rslora**; boolean indicating whether to use rank stablized LoRA (rsLoRA) for potentiallly more stable training



In [ ]:
from datasets import load_dataset
dataset= load_dataset("ServiceNow-AI/R1-Distill-SFT",'v0',split='train')

In [ ]:
print(dataset)

Dataset({
    features: ['id', 'reannotated_assistant_content', 'problem', 'source', 'solution', 'verified', 'quality_metrics'],
    num_rows: 171647
})


In [ ]:
dataset['reannotated_assistant_content'][0]

'<think>\nFirst, I need to determine the total number of children on the playground by adding the number of boys and girls.\n\nThere are 27 boys and 35 girls.\n\nAdding these together: 27 boys + 35 girls = 62 children.\n\nTherefore, the total number of children on the playground is 62.\n</think>\n\nTo find the total number of children on the playground, we simply add the number of boys and girls together.\n\n\\[\n\\text{Total children} = \\text{Number of boys} + \\text{Number of girls}\n\\]\n\nPlugging in the given values:\n\n\\[\n\\text{Total children} = 27 \\text{ boys} + 35 \\text{ girls} = 62 \\text{ children}\n\\]\n\n**Final Answer:**\n\n\\[\n\\boxed{62}\n\\]'

In [ ]:
print(dataset['reannotated_assistant_content'][0])

<think>
First, I need to determine the total number of children on the playground by adding the number of boys and girls.

There are 27 boys and 35 girls.

Adding these together: 27 boys + 35 girls = 62 children.

Therefore, the total number of children on the playground is 62.
</think>

To find the total number of children on the playground, we simply add the number of boys and girls together.

\[
\text{Total children} = \text{Number of boys} + \text{Number of girls}
\]

Plugging in the given values:

\[
\text{Total children} = 27 \text{ boys} + 35 \text{ girls} = 62 \text{ children}
\]

**Final Answer:**

\[
\boxed{62}
\]


In [ ]:
dataset.column_names

['id',
 'reannotated_assistant_content',
 'problem',
 'source',
 'solution',
 'verified',
 'quality_metrics']

In [ ]:
for col in dataset.column_names:
  if col!='id':
    print('---'*10,'\n', col,'\n','---'*10)
    print(dataset[col][20])

------------------------------ 
 reannotated_assistant_content 
 ------------------------------
<think>
First, calculate the number of correct answers Liza got by finding 90% of 60.

Next, determine Rose's correct answers by adding 2 to Liza's correct answers.

Finally, subtract the number of correct answers Rose got from the total number of items to find the number of incorrect answers.
</think>

**Solution:**

1. **Calculate Liza's Correct Answers:**

   Liza got 90% of the 60 items correct.
   
   \[
   \text{Correct Answers (Liza)} = 0.90 \times 60 = 54
   \]

2. **Determine Rose's Correct Answers:**

   Rose got 2 more correct answers than Liza.
   
   \[
   \text{Correct Answers (Rose)} = 54 + 2 = 56
   \]

3. **Find Rose's Incorrect Answers:**

   The total number of items is 60. To find the number of incorrect answers Rose had:
   
   \[
   \text{Incorrect Answers (Rose)} = 60 - 56 = \boxed{4}
   \]

**Answer:** Rose had \(\boxed{4}\) incorrect answers.
------------------------

In [ ]:
# r1_prompt=""" You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
# <problem>
# {}
# </problem>

# {}
# {}
# """

# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#   problems = examples['problem']
#   thoughts = examples["reannotated_assistant_content"]
#   solutions = examples["solution"]
#   texts = []
#   for problem, thought, solution in zip(problems, thoughts, solutions):
#     text = r1_prompt.format(problem, thought, solution)+EOS_TOKEN
#     texts.append(text)

#   return {'text':texts}
# datasets = dataset.map(formatting_prompts_func, batched=True)


In [ ]:
r1_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>

{}
{}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
  problems = examples["problem"]
  thoughts = examples["reannotated_assistant_content"]
  solutions = examples["solution"]
  texts = []

  for problem, thought, solution in zip(problems, thoughts, solutions):
    text = r1_prompt.format(problem, thought, solution)+EOS_TOKEN
    texts.append(text)

  return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset.column_names

['id',
 'reannotated_assistant_content',
 'problem',
 'source',
 'solution',
 'verified',
 'quality_metrics',
 'text']

In [ ]:
for col in dataset.column_names:
  if col!='id':
    print('---'*10,'\n', col,'\n','---'*10)
    print(dataset[col][2])

------------------------------ 
 reannotated_assistant_content 
 ------------------------------
<think>
First, determine how many bags Bianca actually recycled by subtracting the number of bags she did not recycle from the total number of bags she had.

Then, calculate the total points she earned by multiplying the number of recycled bags by the points earned per bag.
</think>

**Solution:**

1. **Determine the number of bags Bianca recycled:**

   Bianca had a total of 17 bags but did not recycle 8 of them. Therefore, the number of bags she actually recycled is:
   
   \[
   17 \text{ bags} - 8 \text{ bags} = 9 \text{ bags}
   \]

2. **Calculate the total points earned:**

   Bianca earns 5 points for each bag she recycles. For 9 bags, the total points earned are:
   
   \[
   9 \text{ bags} \times 5 \text{ points/bag} = 45 \text{ points}
   \]

**Final Answer:**

\[
\boxed{45}
\]
------------------------------ 
 problem 
 ------------------------------
Bianca earned 5 points for each

## **Trainer SETUP**

dataset_text_field ; specifies the field in
the dataset that contains the text data

max_seq_length ; maximum length for the inout data


dataset_num_proc ; number of processes to use for data loading

packing; if True, enables sequence packing (concatenates multiple examples into a single sequence to better utlize tokens)

# **Training Arguments; **

- per_device-train-batchsize ; number of samples
per batch for each device

- gradient accumlation steps; number of steps to acculate gradients before updating model weigths

- warmup steps - number of steps for training steps

- learning rate - learning rate for optimizer

- logging steps - frequenct of logging training progresss

- optim  optimizer type here using an 8 bit version of adamW

- weight decay - ragularization parameter for weight decay

- lr scheduler type - type of learning rate scheduler

- seed - random seed for reproducibility

- report to - intergation for observability tools like wanbd, tensorboard etc

In [ ]:
# pip install -U trl


In [ ]:
# %pip install -U psutil


In [ ]:
from trl import SFTTrainer
from transformers import AutoTokenizer, TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
import psutil


In [ ]:
import unsloth_compiled_cache.UnslothSFTTrainer as ust

ust.psutil = psutil

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processors to use for processing the dataset
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The batch size per GPU/TPU core
        gradient_accumulation_steps = 4, # Number of steps to perform befor each gradient accumulation
        warmup_steps = 5, # Few updates with low learning rate before actual training
        max_steps = 60, # Specifies the total number of training steps (batches) to run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        dataloader_num_workers=2,
        report_to = "none", # Use this for WandB etc for observability
    ),
)

NameError: name 'psutil' is not defined

In [ ]:
trainer_stats = trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """
You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<problem>
{}
</problem>
"""
message= sys_prompt.format("How many r's are present in strawberry")
tokenizer = get_chat_template(
    tokenizer,chat_template ="llama-3.1",
)
FastLanguageModel.for_inference(model) # enable native 2X faster inference

messages= [
    {'role':'user',
     'content':message},
]
inputs= tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt',
).to('cuda')

outputs=model.generate(input_id= inputs,
                       max_new_tokens= 1024,
                       use_cache= True,
                       temperature = 1.5, min_p = 0.1)
response= tokenizer.batch_decode(outputs)

In [ ]:
print(response[0])

In [ ]:
model.save_pretrained('model_1_mk')
tokenizer.save_pretrained('model_2_m')

In [ ]:
model.save_pretrained_gguf('chatGGUF',tokenizer)

In [ ]:
# Running the model via Ollama (OPTIONAL)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)

In [ ]:
print(tokenizer._ollama_modelfile)

In [ ]:

!ollama create unsloth_model -f ./chatGGUF/Modelfile